In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

In [3]:
bank=pd.read_csv('bank-full.csv',sep=';')

In [4]:
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
age          45211 non-null int64
job          45211 non-null object
marital      45211 non-null object
education    45211 non-null object
default      45211 non-null object
balance      45211 non-null int64
housing      45211 non-null object
loan         45211 non-null object
contact      45211 non-null object
day          45211 non-null int64
month        45211 non-null object
duration     45211 non-null int64
campaign     45211 non-null int64
pdays        45211 non-null int64
previous     45211 non-null int64
poutcome     45211 non-null object
y            45211 non-null object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [6]:
bank.drop(['duration','contact'],axis=1,inplace=True)

In [7]:
bank['balance'].describe()

count     45211.000000
mean       1362.272058
std        3044.765829
min       -8019.000000
25%          72.000000
50%         448.000000
75%        1428.000000
max      102127.000000
Name: balance, dtype: float64

In [8]:
bank1=bank[(bank['balance'] <= 1500) & (bank['balance'] >= 0)]

In [9]:
bank1.shape

(30550, 15)

In [10]:
bank2=bank[bank['balance'] > 1500]

In [11]:
bank1.skew()

age          0.700530
balance      0.894885
day          0.129356
campaign     4.891244
pdays        2.632879
previous    49.551401
dtype: float64

In [12]:
bank3=bank[bank['balance'] < 0]

In [13]:
print(bank1.shape)
print(bank2.shape)
print(bank3.shape)


(30550, 15)
(10895, 15)
(3766, 15)


In [14]:
bank1['new_crust']=bank1['previous'].apply(lambda row:1 if (row==0) else 0)

In [15]:
bank1.drop('previous',axis=1,inplace=True)

In [16]:
bank1.skew()

age          0.700530
balance      0.894885
day          0.129356
campaign     4.891244
pdays        2.632879
new_crust   -1.688932
dtype: float64

In [17]:
bank1.drop('pdays',axis=1,inplace=True)

In [18]:
bank1.shape

(30550, 14)

In [19]:
bank1=pd.get_dummies(bank1,drop_first=True)

In [20]:
bank1.shape

(30550, 39)

In [21]:
x=bank1.drop('y_yes',axis=1)
y=bank1.y_yes

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [24]:
from sklearn.preprocessing import StandardScaler

In [25]:
sc=StandardScaler()

In [26]:
x_train_sc=sc.fit_transform(x_train)
x_test_sc=sc.transform(x_test)

In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
lgr=LogisticRegression()

In [29]:
lgr.fit(x_train_sc,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [30]:
y_pred=lgr.predict(x_train_sc)

In [31]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [32]:
print('accuracy: ',accuracy_score(y_train,y_pred))
print('confusion matrix: \n',confusion_matrix(y_train,y_pred))
print('classification report: \n',classification_report(y_train,y_pred))

accuracy:  0.9026420388122516
confusion matrix: 
 [[18861   210]
 [ 1872   442]]
classification report: 
               precision    recall  f1-score   support

           0       0.91      0.99      0.95     19071
           1       0.68      0.19      0.30      2314

    accuracy                           0.90     21385
   macro avg       0.79      0.59      0.62     21385
weighted avg       0.88      0.90      0.88     21385



In [33]:
y_pred1=lgr.predict(x_test_sc)

In [34]:
print('accuracy: ',accuracy_score(y_test,y_pred1))
print('confusion matrix: \n',confusion_matrix(y_test,y_pred1))
print('classification report: \n',classification_report(y_test,y_pred1))

accuracy:  0.9006001091107474
confusion matrix: 
 [[8081   97]
 [ 814  173]]
classification report: 
               precision    recall  f1-score   support

           0       0.91      0.99      0.95      8178
           1       0.64      0.18      0.28       987

    accuracy                           0.90      9165
   macro avg       0.77      0.58      0.61      9165
weighted avg       0.88      0.90      0.87      9165



## outliers data

In [35]:
bank2.head()

,age,job,marital,education,default,balance,housing,loan,day,month,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,5,may,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,5,may,1,-1,0,unknown,no
34,51,management,married,tertiary,no,10635,yes,no,5,may,1,-1,0,unknown,no
40,44,services,divorced,secondary,no,2586,yes,no,5,may,1,-1,0,unknown,no
65,51,management,married,tertiary,no,6530,yes,no,5,may,1,-1,0,unknown,no


In [36]:
bank2.drop('previous',axis=1,inplace=True)

In [37]:
bank2.skew()

age         0.618777
balance     6.041180
day         0.005871
campaign    5.003136
pdays       2.569069
dtype: float64

In [38]:
bank2.drop('pdays',axis=1,inplace=True)

In [39]:
bank2.shape

(10895, 13)

In [40]:
bank2=pd.get_dummies(bank2,drop_first=True)

In [41]:
bank2.shape

(10895, 38)

In [42]:
x1=bank2.drop('y_yes',axis=1)
y1=bank2.y_yes

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
x_train1,x_test1,y_train1,y_test1=train_test_split(x1,y1,test_size=0.3,random_state=42)

In [45]:
from sklearn.preprocessing import StandardScaler

In [46]:
sc=StandardScaler()

In [47]:
x_train1_sc=sc.fit_transform(x_train1)
x_test1_sc=sc.transform(x_test1)

In [48]:
from sklearn.linear_model import LogisticRegression

In [49]:
lg=LogisticRegression()

In [50]:
lg.fit(x_train1_sc,y_train1)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [51]:
y_pred2=lg.predict(x_train1_sc)

In [52]:
print('accuracy: ',accuracy_score(y_train1,y_pred2))
print('confusion matrix: \n',confusion_matrix(y_train1,y_pred2))
print('classification report: \n',classification_report(y_train1,y_pred2))

accuracy:  0.8432992394440073
confusion matrix: 
 [[6200  154]
 [1041  231]]
classification report: 
               precision    recall  f1-score   support

           0       0.86      0.98      0.91      6354
           1       0.60      0.18      0.28      1272

    accuracy                           0.84      7626
   macro avg       0.73      0.58      0.60      7626
weighted avg       0.81      0.84      0.81      7626



In [53]:
y_pred3=lg.predict(x_test1_sc)

In [54]:
print('accuracy: ',accuracy_score(y_test1,y_pred3))
print('confusion matrix: \n',confusion_matrix(y_test1,y_pred3))
print('classification report: \n',classification_report(y_test1,y_pred3))

accuracy:  0.8611196084429489
confusion matrix: 
 [[2693   70]
 [ 384  122]]
classification report: 
               precision    recall  f1-score   support

           0       0.88      0.97      0.92      2763
           1       0.64      0.24      0.35       506

    accuracy                           0.86      3269
   macro avg       0.76      0.61      0.64      3269
weighted avg       0.84      0.86      0.83      3269



### pca

In [55]:
from sklearn.decomposition import PCA
pca=PCA(n_components=15)

In [56]:
bank=pd.get_dummies(bank,drop_first=True)

In [57]:
bank.columns

Index(['age', 'balance', 'day', 'campaign', 'pdays', 'previous',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_married', 'marital_single', 'education_secondary',
       'education_tertiary', 'education_unknown', 'default_yes', 'housing_yes',
       'loan_yes', 'month_aug', 'month_dec', 'month_feb', 'month_jan',
       'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov',
       'month_oct', 'month_sep', 'poutcome_other', 'poutcome_success',
       'poutcome_unknown', 'y_yes'],
      dtype='object')

In [58]:
x1=bank.drop('y_yes',axis=1)
y1=bank.y_yes

In [59]:
x1_train,x1_test,y1_train,y1_test=train_test_split(x1,y1,test_size=0.3,random_state=42)

In [60]:
sc=StandardScaler()

In [61]:
x1_train_sc=sc.fit_transform(x1_train)
x1_test_sc=sc.transform(x1_test)

In [62]:
x1_train_pca=pca.fit_transform(x1_train_sc)
x1_test_pca=pca.transform(x1_test_sc)

In [63]:
lg1=LogisticRegression()

In [64]:
lg1.fit(x1_train_pca,y1_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [65]:
y_pred_p=lg1.predict(x1_train_pca)

In [66]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [67]:
print('accuracy: ',accuracy_score(y1_train,y_pred_p))
print('confusion matrix: \n',confusion_matrix(y1_train,y_pred_p))
print('classification report: \n',classification_report(y1_train,y_pred_p))

accuracy:  0.8897841817549846
confusion matrix: 
 [[27617   339]
 [ 3149   542]]
classification report: 
               precision    recall  f1-score   support

           0       0.90      0.99      0.94     27956
           1       0.62      0.15      0.24      3691

    accuracy                           0.89     31647
   macro avg       0.76      0.57      0.59     31647
weighted avg       0.86      0.89      0.86     31647



In [68]:
y_pred_t=lg1.predict(x1_test_pca)

In [69]:
print('accuracy: ',accuracy_score(y1_test,y_pred_t))
print('confusion matrix: \n',confusion_matrix(y1_test,y_pred_t))
print('classification report: \n',classification_report(y1_test,y_pred_t))

accuracy:  0.8894868770274256
confusion matrix: 
 [[11817   149]
 [ 1350   248]]
classification report: 
               precision    recall  f1-score   support

           0       0.90      0.99      0.94     11966
           1       0.62      0.16      0.25      1598

    accuracy                           0.89     13564
   macro avg       0.76      0.57      0.59     13564
weighted avg       0.87      0.89      0.86     13564



In [70]:
x1_test.shape

(13564, 39)